<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [219]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [220]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [221]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [225]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [0]:
import keras
from keras.utils import np_utils


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [231]:
ldat["rank"] 

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
          ... 
542850     9.0
542851    10.0
542852    11.0
542853    12.0
542854    13.0
Name: rank, Length: 542855, dtype: float64

In [0]:
# 前回の3位以内にマークを付ける
ldat["prev_high_rank"] = 0
ldat.loc[ldat["prev_rank"] <= 3, "prev_high_rank"] = 1

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 前回ランクを段階化
ldat["prev_rank_level"] = round(ldat["prev_rank"]  / 4)

In [0]:
# ランクを段階化
ldat["rank_level"] = round(ldat["rank"]  / 4)

In [0]:
# 人気を段階化
ldat["prev_popularity_level"] = round(ldat["prev_popularity"] / 4)

In [0]:
# スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# スピードを段階化
ldat["prev_speed_level"] =  round(ldat["prev_speed"]*10000000)

In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True).dropna(0)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("prev_high_rank == 1")

In [0]:
loser = sdat.query("prev_high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
# x , yデータを作る start

In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
x = sampled[['prev_rank_level',"prev_speed_level"]].dropna(0)

# lastx_tmpは最終評価用に使用、勝ち負けを半々にしていない
lastx_tmp = sdat[['prev_rank_level',"prev_speed_level"]].dropna(0)

In [0]:
#x = sampled[['high_prev_popularity']].dropna(0)

In [0]:
#y = sampled[["high_rank"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["high_rank"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"],index=sampled.index)

# lasty_tmpは最終評価用に使用、勝ち負けを半々にしていない
lasty_np = np_utils.to_categorical(sdat[["high_rank"]].dropna(0))
lasty_tmp = pd.DataFrame(lasty_np,columns=["lose","win"],index=sdat.index)

In [254]:
len(lasty)

352595

In [255]:
sampled["high_rank"]

287153    0
81315     0
54302     1
541272    0
427385    0
         ..
109579    1
69901     1
59150     0
2525      0
460520    0
Name: high_rank, Length: 208460, dtype: int64

In [0]:
# x , yデータを作る end

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
testorg = sampled.tail(len(x)-trainlen) 

In [0]:
# 学習、テストデータの作成　終わり

In [264]:
testx.loc[542846]

KeyError: ignored

In [0]:
# 最終検証用データからトレーニングデータを取り除く
lastx = lastx_tmp[~lastx_tmp.index.isin(trainx.index)]
lasty = lasty_tmp[~lasty_tmp.index.isin(trainy.index)]
lastsdat = sdat[~sdat.index.isin(trainx.index)]

In [266]:
lastsdat

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
12,3.0,6,9,1074,1,4,56.0,83,1:10.1,7,6.5,4.0,478.0,4.0,143,12,9,0,0,-,1200.0,3,1,16:00,2007,4,22,2.0,6.0,12.0,1074.0,1.0,4.0,56.0,97.0,1:10.2,9.0,6.6,4.0,474.0,-8.0,143.0,12.0,9.0,0.0,0.0,-,1200.0,2.0,0.0,16:05,2007.0,3.0,31.0,False,0.0,1561165812,1561219200,1561219500,1,1,0.0,1.0,1.0,7.686563e-07,8.0
16,16.0,4,7,6051,2,7,52.0,27,2:27.3,1,216.8,16.0,458.0,4.0,47,10,9,0,2,-,2400.0,0,0,15:00,2008,10,12,11.0,5.0,8.0,6051.0,2.0,7.0,57.0,85.0,2:01.6,17.0,200.1,14.0,454.0,-2.0,47.0,10.0,9.0,0.0,0.0,-,2000.0,0.0,0.0,15:10,2008.0,9.0,14.0,False,0.0,1561168896,1561215600,1561216200,0,0,3.0,4.0,4.0,1.281091e-06,13.0
17,11.0,2,4,17463,1,4,57.0,157,1:47.8,14,166.0,15.0,460.0,-8.0,18,12,9,1,2,-,1700.0,0,0,16:40,2009,5,24,10.0,2.0,3.0,17463.0,1.0,4.0,57.0,201.0,1:11.8,9.0,58.5,11.0,468.0,12.0,18.0,12.0,9.0,1.0,0.0,-,1200.0,2.0,3.0,16:40,2009.0,4.0,25.0,False,0.0,1561165908,1561221600,1561221600,0,0,2.0,3.0,3.0,7.686563e-07,8.0
18,9.0,6,8,31765,0,5,54.0,7,1:20.7,3,9.0,4.0,488.0,-4.0,593,11,2,0,0,-,1400.0,0,0,15:35,2014,4,12,1.0,4.0,8.0,31765.0,0.0,5.0,55.0,7.0,1:21.2,0.0,2.2,1.0,492.0,-2.0,238.0,11.0,9.0,0.0,0.0,外,1400.0,0.0,0.0,15:45,2014.0,1.0,26.0,False,0.0,1561166472,1561217700,1561218300,1,0,0.0,2.0,0.0,8.967654e-07,9.0
21,15.0,8,18,36894,1,6,57.0,145,1:24.2,4,111.7,14.0,460.0,-2.0,449,12,9,0,2,-,1400.0,0,0,16:10,2016,3,21,9.0,8.0,14.0,36894.0,1.0,6.0,57.0,321.0,1:48.9,14.0,54.8,11.0,462.0,0.0,449.0,7.0,9.0,0.0,0.0,-,1800.0,0.0,1.0,13:05,2016.0,2.0,21.0,False,0.0,1561168134,1561219800,1561208700,0,0,2.0,4.0,3.0,1.152983e-06,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542849,12.0,8,16,5388,0,3,54.0,85,1:37.2,4,278.9,15.0,422.0,2.0,94,5,9,0,0,外,1600.0,1,0,12:20,2007,3,17,10.0,3.0,6.0,5388.0,0.0,2.0,54.0,121.0,1:42.8,1.0,222.1,12.0,420.0,0.0,94.0,1.0,9.0,1.0,2.0,-,1600.0,2.0,0.0,09:50,2006.0,11.0,19.0,False,0.0,1561167768,1561206000,1561197000,0,0,2.0,3.0,3.0,1.024874e-06,10.0
542851,5.0,7,14,41689,1,3,54.0,253,2:01.9,4,4.2,2.0,508.0,4.0,226,7,9,0,0,-,2000.0,0,0,13:15,2014,9,28,17.0,8.0,18.0,41689.0,1.0,3.0,56.0,5.0,2:31.5,13.0,18.6,6.0,504.0,0.0,226.0,11.0,2.0,0.0,2.0,-,2400.0,0.0,0.0,15:45,2014.0,5.0,3.0,False,0.0,1561170690,1561209300,1561218300,0,0,4.0,1.0,2.0,1.537308e-06,15.0
542852,10.0,3,6,7575,1,6,56.0,194,1:51.2,6,56.7,9.0,502.0,0.0,237,10,9,1,0,-,1800.0,0,0,15:35,2008,7,12,9.0,2.0,2.0,7575.0,1.0,6.0,57.0,194.0,1:49.5,6.0,25.6,8.0,502.0,4.0,237.0,12.0,9.0,0.0,0.0,外,1800.0,1.0,2.0,16:50,2008.0,6.0,29.0,False,0.0,1561168170,1561217700,1561222200,0,0,2.0,2.0,2.0,1.152983e-06,12.0
542853,5.0,7,11,20044,1,3,56.0,44,1:56.7,4,5.7,4.0,472.0,-4.0,16,3,9,1,2,-,1800.0,1,0,10:40,2009,5,17,4.0,2.0,2.0,20044.0,1.0,3.0,56.0,44.0,1:53.5,1.0,15.4,8.0,476.0,-8.0,16.0,1.0,9.0,1.0,0.0,-,1800.0,3.0,3.0,10:30,2009.0,4.0,25.0,False,0.0,1561168410,1561200000,1561199400,0,0,1.0,1.0,2.0,1.152983e-06,12.0


In [267]:
lasty

,lose,win
12,0.0,1.0
16,1.0,0.0
17,1.0,0.0
18,1.0,0.0
21,1.0,0.0
...,...,...
542849,1.0,0.0
542851,1.0,0.0
542852,1.0,0.0
542853,1.0,0.0


In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 5


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [273]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               1536      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [275]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 104230 samples, validate on 104230 samples
Epoch 1/5
104230/104230 [==============================] - 8s 81us/step - loss: 0.5512 - accuracy: 0.7260 - val_loss: 0.5567 - val_accuracy: 0.7246
Epoch 2/5
104230/104230 [==============================] - 8s 78us/step - loss: 0.5473 - accuracy: 0.7267 - val_loss: 0.5511 - val_accuracy: 0.7246
Epoch 3/5
104230/104230 [==============================] - 8s 76us/step - loss: 0.5472 - accuracy: 0.7267 - val_loss: 0.5489 - val_accuracy: 0.7246
Epoch 4/5
104230/104230 [==============================] - 8s 76us/step - loss: 0.5467 - accuracy: 0.7267 - val_loss: 0.5480 - val_accuracy: 0.7246
Epoch 5/5
104230/104230 [==============================] - 8s 76us/step - loss: 0.5465 - accuracy: 0.7267 - val_loss: 0.5480 - val_accuracy: 0.7246


In [0]:
# 予測
predictions = model.predict(lastx)

In [277]:
print(predictions)

[[0.6133012  0.3841424 ]
 [0.91210073 0.09229252]
 [0.84710085 0.15719235]
 ...
 [0.8360472  0.16814351]
 [0.6265723  0.37427762]
 [0.8471009  0.15719232]]


In [278]:
# 評価
score = model.evaluate(lastx, lasty, verbose=1)


352595/352595 [==============================] - 12s 35us/step


In [279]:
print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.45968444636424266
Test accuracy 0.8060352802276611


In [0]:
# 回収率の計算

In [0]:
## 予想結果ともとのデータを結合

In [0]:
predict = pd.DataFrame(predictions, columns={"lose","win"},index=lastx.index)

In [0]:
result =  pd.merge(predict, lastsdat, right_index=True, left_index=True)

In [0]:
# リターンと支払額を計算

In [0]:
return_amount = result.query("win > 0.3")[["return"]].sum()["return"]
pay_amount =result.query("win > 0.3").count()["lose"]

In [340]:
# 最終損益
return_amount - pay_amount 

-25874.20000000001

In [0]:
# 的中率の計算

In [0]:
# 勝率が高いと予想し、かつ実際に3位以内に入った馬
success = result.query("win > 0.4 and rank <= 3").count()["lose"]

# 勝率が高いと予想した馬の合計数
total = result.query("win > 0.4").count()["lose"]

In [348]:
# 的中率
success / total

0.4112903225806452

124

In [0]:
# 競馬予想終わり

In [333]:
result.query("win > 0.3 and rank <= 3").count()["lose"]

37318

110822